In [1]:
import csadata.twitter as twitter
from datetime import datetime, timezone

In [2]:
start = int(datetime(2020, 10, 1, 0, 0, 0, tzinfo=timezone.utc).timestamp())
end = int(datetime(2021, 7, 14, 23, 59, 59, tzinfo=timezone.utc).timestamp())

input_path = None
output_path = None

reader = twitter.TweetCSVReader(path=input_path, start=start, end=end)
writer = twitter.TweetCSVWriter(path=output_path)

duplicate_filter = twitter.TweetDuplicateFilter(dissimilarity_threshold=.2, batch_size=200)

for tweet in reader:
    twitter.preprocess_text(tweet)

    if duplicate_filter.feed(tweet):
        filtered_tweet = None

        while True:
            filtered_tweet = duplicate_filter.pull()

            if filtered_tweet is None:
                break
            else:
                writer.write(filtered_tweet)

print("Processed Tweets: {}".format(duplicate_filter.processed_tweets))
print("Eliminated Tweets: {}".format(duplicate_filter.eliminated_tweets))
print("Tweet duplicate ratio: {:.2f} %".format(duplicate_filter.eliminated_tweets / duplicate_filter.processed_tweets * 100))

Processed Tweets: 8851886
Eliminated Tweets: 317399
Tweet duplicate ratio: 3.59 %
